In [1]:
# financial_behavior_simulation.py
import pandas as pd
import numpy as np
import os
import random

# ---------- Setup ----------
DOWNLOADS_DIR = os.path.join(os.path.expanduser("~"), "Downloads", "synthetic_data")
CUSTOMERS_PATH = os.path.join(DOWNLOADS_DIR, "customers_hma_10000.csv")  # or customers_ctgan_10000.csv

# Load customer data
customers = pd.read_csv(CUSTOMERS_PATH)
customers = customers.head(5000)  # for testing; remove limit for full run

# Personas
personas = ['Saver', 'Spender', 'Investor', 'Risk-Taker']
customers['persona'] = np.random.choice(personas, size=len(customers))

# Simulation parameters
n_months = 12
records = []

for _, row in customers.iterrows():
    cust_id = row['customer_id']
    income_base = row['income'] / 12 if 'income' in row else np.random.randint(30000, 120000)
    balance = row.get('account_balance', np.random.randint(20000, 100000))
    persona = row['persona']

    for month in range(1, n_months + 1):
        # Income varies slightly each month
        income = income_base * np.random.uniform(0.9, 1.1)

        # Persona-based spending pattern
        if persona == 'Saver':
            spending = income * np.random.uniform(0.3, 0.5)
            investment = income * np.random.uniform(0.1, 0.2)
        elif persona == 'Spender':
            spending = income * np.random.uniform(0.7, 0.9)
            investment = income * np.random.uniform(0.0, 0.05)
        elif persona == 'Investor':
            spending = income * np.random.uniform(0.5, 0.7)
            investment = income * np.random.uniform(0.2, 0.3)
        else:  # Risk-Taker
            spending = income * np.random.uniform(0.4, 0.9)
            investment = income * np.random.uniform(0.0, 0.4)

        # Loan repayment (if they have an existing loan)
        loan_repayment = 0
        if 'existing_loan_balance' in row and row['existing_loan_balance'] > 0:
            loan_repayment = min(row['existing_loan_balance'] / 120, income * 0.15)  # monthly EMI

        # Savings = income - total expenses
        savings = income - (spending + investment + loan_repayment)
        balance += savings

        # Save monthly record
        records.append({
            'customer_id': cust_id,
            'month': month,
            'persona': persona,
            'income': round(income, 2),
            'spending': round(spending, 2),
            'savings': round(savings, 2),
            'investments': round(investment, 2),
            'loan_repayment': round(loan_repayment, 2),
            'balance': round(balance, 2)
        })

# ---------- Save ----------
financial_behavior = pd.DataFrame(records)
out_path = os.path.join(DOWNLOADS_DIR, "financial_behavior.csv")
financial_behavior.to_csv(out_path, index=False)
print(f"✅ Financial behavior simulation complete! Saved at:\n{out_path}")

✅ Financial behavior simulation complete! Saved at:
C:\Users\user\Downloads\synthetic_data\financial_behavior.csv
